In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir) 
sys.path.append(parentdir)

In [2]:
import requests
import re
import pandas as pd

from bs4 import BeautifulSoup


response = requests.get('https://cityclerk.lacity.org/lacityclerkconnect/index.cfm?fa=ccfi.viewrecord&cfnumber=21-1247')
soup = BeautifulSoup(response.text, "html.parser")

In [19]:
import json
from scrapper.transform_load import table_cleaner, DataBase

with open('extracted_raw/LaCLerk_spider_results.json') as f:
    json_=json.load(f)

frames = table_cleaner(json_[1])
db = DataBase()

db.update_from_frame(frames[0], 'actions')
db.update_from_frame(frames[1], 'documents')
db.update_from_frame(frames[2], 'summary')
db.update_from_frame(frames[3], 'vote')
db.update_from_frame(frames[4], 'members')
db.update_from_frame(frames[5], 'actions_documents')


Connecting to 'storage.db'


7

In [24]:
conn= db.conn

pd.read_sql_table('summary', conn)

,file_id,url,date_received,last_modified,expiration,reference,district,initiated_date,title,initiated_by
0,14-0694,https://cityclerk.lacity.org/lacityclerkconnec...,2014-05-27,2014-08-01,2016-07-30,None,15,NaT,L Street / Lecouvreur Street / Zone Change App...,


In [10]:
import pandas as pd


pd.read_sql_table('documents', db.conn)

,file_id,name,url,date,actions_index


In [7]:
db.conn.engine.table_names()

<ipython-input-7-9f6e73dff187>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  db.conn.engine.table_names()


['actions', 'documents', 'members', 'summary', 'vote']

In [27]:
from sqlalchemy import create_engine
from scrapper.constants import ACTIONS_TABLE_SQL

engine = create_engine('sqlite:///test.db')
conn = engine.connect()
conn.execute(ACTIONS_TABLE_SQL)

In [28]:
conn.engine.table_names()
conn.close()

<ipython-input-28-25655b114a98>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  conn.engine.table_names()


In [129]:
find_elements_by_text(soup, 'File History')

"6-6-06 - This day's Council session 6-6-06 - Ref to Planning and Land Use Management Committee 6-6-06 - File to Planning and Land Use Management Committee Clerk 8-11-06 - Set for Planning and Land Use Management Committee on August 15, 2006 8-15-06 - Planning and Land Use Management Committee Disposition - Refer to staff 8-16-06 - Letter to Planning and Building and Safety Departments and City Attorney per Planning and Land Use Management Committee Clerk 7-11-07 - For ref - Communication from the City Planning Commission, dated July 9, 2007, relative to a proposed Baseline Mansionization Ordinance applicable to all properties zoned single-family residential (R1, RS, RE9, RE11, RE15, RA, RE20, and RE40), not located in Hillside Area or Coastal Zone, that amends various sections of the Los Angeles Municipal Code (LAMC) including Sections 12.03, Subsection C of Sections 12.07, 12.07.01, 12.07.1, 12.08, Section 12.21.1\nA, and Section 12.28\nA / CPC 2007-106-CA. 7-12-07 - Ref to Planning 

In [150]:
from scrapper.spider import get_all, get_vote_info
get_all(soup, 1)

{'id': 1,
 'title': '4800 Block of Oak Park Avenue / Plan Amendment and Zone Change',
 'date_received': '2010-01-29',
 'last_modified': '2012-07-09',
 'expiration': '2012-02-01',
 'references': '',
 'district': '5',
 'initiated_by': '',
 'movers': ['paul koretz'],
 'seconds': ['dennis zine'],
 'vote_data': {},
 'actions': [{'date': '2012-07-09',
   'activity': 'File expired per Council policy, Council file No. 05-0553.',
   'documents': [{'name': 'Council Action',
     'url': 'https://clkrep.lacity.org/onlinedocs/2010/10-0180_ca_7-9-2012.pdf',
     'date': '2012-07-09'}]},
  {'date': '2010-01-29',
   'activity': 'Motion referred to Planning and Land Use Management Committee.',
   'documents': [{'name': 'Motion',
     'url': 'https://clkrep.lacity.org/onlinedocs/2010/10-0180_mot_1-29-10.pdf',
     'date': '2010-01-29'}]}],
 'documents': [{'name': 'Council Action',
   'url': 'https://clkrep.lacity.org/onlinedocs/2010/10-0180_ca_7-9-2012.pdf',
   'date': '2012-07-09'},
  {'name': 'Motion'